# SPS Hack Night I - Spring 2020 

## An Introduction to GPU Programming 

GPUs are incredibly efficient at highly parallel operations.

Since Monte Carlo generation is a series of independent operations, we can make it more efficient through the use of a GPU.

------

# Numba

Let's import numba, the package we will be using to speed up calculations via parallel processing, and check if it sees the GPU.

**Note:** doing this on a laptop/desktop with a CUDA-capable Nvidia GPU is an incredibly long but rewarding setup process. I highly reccomend the use of conda environments to streamline the process.


In [0]:
from numba import cuda

print(cuda.is_available())

Now let's write some code to find pi via a Monte Carlo Simulation. We will generate an array of random numbers for x and y, then check if the magnitude of (`x[i]`, `y[i]`) is greater than or less than 1.

In [0]:
import numpy as np
import time
import numba 

def calc_pi(n):
    np.random.seed(0)
    x = 2*np.random.ranf(n)-1
    y = 2*np.random.ranf(n)-1
    return 4*np.sum(x**2+y**2<1)/n

points = 200000000

t1 = time.time()
pi = calc_pi(points)
selftimed = time.time()-t1
print("SELFTIMED ", selftimed)
print("result: ", pi)


We can quite simply parallelize this code using numba. The speedup is huge! Well, not that huge...

In [0]:
import numba

run_parallel = numba.config.NUMBA_NUM_THREADS > 1

@numba.njit(parallel=run_parallel)
def calc_pi_parallel(n):
    np.random.seed(0)
    x = 2*np.random.ranf(n)-1
    y = 2*np.random.ranf(n)-1
    return 4*np.sum(x**2+y**2<1)/n

t1 = time.time()
pi = calc_pi_parallel(points)
selftimed = time.time()-t1
print("SELFTIMED Parallel ", selftimed)
print("Parallel Result: ", pi)

This is cool and all, but we can do better.

Act II: Enter CUDA.

-------

## Using CUDA with Numba

This code takes advantage of the GPU for huuuuggggeeeee results.

In [0]:
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

points = 200000000000

@cuda.jit
def compute_pi(rng_states, iterations, out):
    thread_id = cuda.grid(1)
    inside = 0
    for i in range(iterations):
        x = xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = xoroshiro128p_uniform_float32(rng_states, thread_id)
        if x**2 + y**2 <= 1.0:
            inside += 1

    out[thread_id] = 4.0 * inside / iterations

threads_per_block = 64
blocks = 24
iterations = int(math.ceil(points / threads_per_block / blocks))

t1 = time.time()
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=0)
out = np.zeros(threads_per_block * blocks, dtype=np.float32)
compute_pi[blocks, threads_per_block](rng_states, iterations, out)
pi = out.mean()
selftimed = time.time()-t1

print("SELFTIMED GPU ", selftimed)
print("GPU Result: ", pi)

Look at this awesome result! We just used a GPU to significantly speed up the caclulation of pi. What else can we do?

------


## The Monty Hall Problem

Suppose you are on a game show with three doors. Behind two of the doors are goats. Behind the third is a shiny new car. You select one of the doors. The host opens a different door revealing that it contains a goat, and asks you if you would like to switch your pick. Should you?

Let's use a Monte Carlo simulation to find out.

In [0]:
import numpy as np
import time
import numba 
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import math

trials = 2**24
switch_guess = True

@cuda.jit
def sim(rng_states, iterations, out):
    thread_id = cuda.grid(1)
    
    correct = 0
    for i in range(iterations):
        correct_answer = int(math.floor(3*xoroshiro128p_uniform_float32(rng_states, thread_id)))
        guess = int(math.floor(3*xoroshiro128p_uniform_float32(rng_states, thread_id)))

        open_door = 0
        while True:
            open_door = int(math.floor(3*xoroshiro128p_uniform_float32(rng_states, thread_id)))
            if open_door != guess and open_door != correct_answer:
                break
        
        if switch_guess:
            for a in range(3):
                if a!=guess and a!=open_door:
                    final_guess = a
                    break
        else:
            final_guess = guess

        if correct_answer == final_guess:
            correct += 1

    out[thread_id] = correct / iterations


threads_per_block = 64
blocks = 24
iterations = int(math.ceil(trials / threads_per_block / blocks))

t1 = time.time()
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=0)
out = np.zeros(threads_per_block * blocks, dtype=np.float32)
sim[blocks, threads_per_block](rng_states, iterations, out)
percent_correct = out.mean()
selftimed = time.time()-t1

print("SELFTIMED GPU ", selftimed)
print("GPU Result: ", percent_correct)